In [2]:
import sys
sys.path.append("..")
sys.path.append("../nl4opt-subtask2-baseline")

import parsers
import jsonlines
import prettyprinter as pp

pp.install_extras(exclude=['python', 'django'])

In [7]:
json_parser = parsers.JSONFormulationParser()
xml_parser = parsers.ModelOutputXMLParser()
# regular test pipeline of a few examples
with jsonlines.open('./data/train.jsonl') as reader:
    true_formulations = []

    for i, line in enumerate(reader.iter()):
        true_formulation = json_parser.parse(line)
        true_formulations.append(true_formulation)
        # make sure you use the order mapping from the ground truth (true_formulation.entities)
        if i == 0:
            print(line[list(line.keys())[0]]['obj_declaration']['direction'])
            print(line[list(line.keys())[0]]['document'])
pp.pprint(true_formulations[0])

for entity in true_formulations[0].entities:
    print(entity, true_formulations[0].entities[entity])

minimize
A hotel employs cleaners and receptionists. Cleaners earn $500 per week and receptionists earn $350 per week. The hotel requires a minimum of 100 workers of whom at least 20 must be receptionists. To keep the hotel clean and running smoothly, the number of receptionists should be at least a third of the number of cleaners. The hotel wants to keep the weekly wage bill below $30000. Formulate a LP to minimize the wage bill.
parsers.ProblemFormulation(
    objective=parsers.ObjectiveDeclaration(
        direction='minimize',
        terms={
            'Cleaners': parsers.Term(name='Cleaners', index=0, value=500.0),
            'receptionists': parsers.Term(
                name='receptionists',
                index=1,
                value=350.0
            )
        },
        entities={
            'cleaners': 0,
            'receptionists': 1,
            'Cleaners': 0
        },
        name='wage bill'
    ),
    constraints=[
        parsers.ConstraintDeclaration(
       

# json to canonical form

In [4]:
t1 = parsers.convert_to_canonical(true_formulations[1])
print(t1.objective)
print(t1.constraints)

CanonicalFormulation(objective=array([0.3, 0.1]), constraints=array([[ 1.0e+00,  1.0e+00,  5.0e+04],
       [-0.0e+00, -1.0e+00, -1.0e+04],
       [ 2.4e+01,  2.5e+01, -0.0e+00]]))
[0.3 0.1]
[[ 1.0e+00  1.0e+00  5.0e+04]
 [-0.0e+00 -1.0e+00 -1.0e+04]
 [ 2.4e+01  2.5e+01 -0.0e+00]]


# canonical form to nature language

In [5]:
var_num = max(len(t1.objective), len(t1.constraints[0])- 1)
print(f"{var_num = }")

def format(var_id, coef):
    ans = ""
    if var_id == 0:
        ans = f"{coef}x_{var_id}"
        return ans

    if coef < 0:
        ans += f" - {-coef}x_{var_id}"
    else:
        ans += f" + {coef}x_{var_id}"
    return ans

# objective function
output = "minimize:\n"

for i, coef in enumerate(t1.objective):
    coef = round(coef, 2)
    # -0.0 -> 0.0
    if coef == -0.0:
        coef = 0.0

    output += format(i, coef)

# constrains
output += "\nsubject to:\n"

for constraint in t1.constraints:
    for i, coef in enumerate(constraint):
        coef = round(coef, 2)
        # -0.0 -> 0.0
        if coef == -0.0:
            coef = 0.0

        if i <= len(constraint) - 2:
            output += format(i, coef)
        else:
            output += f" <= {coef}\n"

print(output)

var_num = 2
minimize:
350.0x_0 + 75.0x_1
subject to:
8.0x_0 + 2.0x_1 <= 500.0
0.7x_0 - 0.3x_1 <= 0.0
1000.0x_0 + 150.0x_1 <= 20000.0

